# Preentrenamiento del modelo especializado en matrículas.

In [1]:
from ultralytics import YOLO

model = YOLO('yolo11n.pt')

results = model.train(
  data="data.yaml",  # <-- CAMBIA ESTA LÍNEA
  epochs=20,
  imgsz=640,
  batch=4
)

New https://pypi.org/project/ultralytics/8.3.218 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.215  Python-3.9.23 torch-2.8.0+cpu CPU (AMD Ryzen 7 5700U with Radeon Graphics)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train12, nbs=64, nms=False, opset=None, optim

### Carga del modelo entrenado

In [2]:
# En caso de ejecutar con un modelo ya entrenado
from ultralytics import YOLO 

model = YOLO('runs/detect/train12/weights/best.pt')
